In [1]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
from datetime import datetime
import random

from tqdm import tqdm

from xgboost import XGBRegressor, XGBRanker
#from sklearn.xgboost import XGBRegressor

from sklearn.ensemble import HistGradientBoostingRegressor, GradientBoostingRegressor, HistGradientBoostingClassifier, AdaBoostRegressor
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.model_selection import cross_val_score, RepeatedStratifiedKFold
from sklearn.metrics import mean_squared_error, ndcg_score
from sklearn.utils import resample
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingRandomSearchCV, GroupKFold

In [2]:
def score_predict(X_test, y_test, pred):

    df = X_test.copy()
    df['true_y'] = y_test
    df['pred_y'] = pred

    match = sum((df.groupby('srch_id')['true_y'].idxmax() == df.groupby('srch_id')['pred_y'].idxmax()) & (df.groupby('srch_id')['true_y'].max() > 1))

    score = match / df['srch_id'].nunique()
    return score

In [ ]:
# data = pd.read_csv('data/train_train.csv')

train_data = pd.read_csv('data/train_train_FE.csv')
train_data.drop('Unnamed: 0', axis=1, inplace=True)

train_data.drop('date_time', axis = 1, inplace=True)
#train_data.drop('prob_searchclick', axis = 1, inplace=True)

train_data['target'] = train_data['click_bool'] + 4* train_data['booking_bool']
train_data['target'].loc[train_data['target'].isna()] = int(0)

In [ ]:
test_data = pd.read_csv('data/train_test_FE.csv')
test_data.drop('Unnamed: 0', axis=1, inplace=True)

test_data.drop('date_time', axis = 1, inplace=True)
test_data['target'] = test_data['click_bool'] + 4* test_data['booking_bool']
test_data['target'].loc[test_data['target'].isna()] = int(0)

In [ ]:
train_data['target'].hist()
plt.yscale('log')

In [ ]:
# data.sort_values(['srch_id', 'target'], axis=0, inplace=True, ignore_index=True, ascending=[True, False])
# ids = data['srch_id'].unique()

In [ ]:
# train_ids = random.sample(list(ids), k = int(0.95* len(ids)))
# test_ids = set(ids) - set(train_ids)

In [ ]:
# data = data.join(data.loc[data['srch_id'].isin(train_ids)].groupby('prop_id')['booking_bool'].mean(), on = 'prop_id', rsuffix = '_mean')
# data = data.join(data.loc[data['srch_id'].isin(train_ids)].groupby('prop_id')['click_bool'].mean(), on = 'prop_id', rsuffix = '_mean')

In [ ]:
# train_data = data.loc[data['srch_id'].isin(train_ids)]
# test_data = data.loc[data['srch_id'].isin(test_ids)]

In [ ]:
# # get search ids with any booking
# ids_train = train_data['srch_id'].loc[train_data['booking_bool'] == 1].unique()
# # use only the searches that resulted in a booking
# train_data = train_data.loc[train_data['srch_id'].isin(ids)]
# # only use properties that are being booked
# props = train_data['prop_id'].loc[train_data['booking_bool'] == 1].unique()
# data = train_data.loc[train_data['prop_id'].isin(props)]

In [ ]:
#undersample non bookings
X_under, y_under = resample(train_data.drop(['true_rank', 'click_bool', 'gross_bookings_usd', 'booking_bool', 'position', 'target']
                                      , axis=1).loc[train_data['target'] < 1], train_data['target'].loc[train_data['target'] < 1]
                                      , n_samples= 1* sum(data['target'] >= 1), replace=False)

In [ ]:
# X_bal = pd.concat([X_under, train_data.drop(['true_rank', 'click_bool', 'gross_bookings_usd', 'booking_bool', 'position', 'target']
#                                       , axis=1).loc[train_data['target'] >= 1]])

# y_bal = y_under
# y_bal = y_bal.append(train_data['target'].loc[train_data['target'] >= 1])

X_bal = train_data.drop(['click_bool', 'gross_bookings_usd', 'booking_bool', 'position', 'target'], axis=1)
y_bal = train_data['target']


In [ ]:
plt.hist(y_bal)

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(X_bal, y_bal)

# X_train, X_test, y_train, y_test = train_test_split(data.drop(['booking_bool_mean', 'click_bool_mean', 'click_bool', 'gross_bookings_usd', 'booking_bool', 'position', 'target']
#                                        , axis=1), data['target'])
#X_train, y_train = X_bal, y_bal

In [ ]:
# much better than just replacing with 0
# X_train = X_train.fillna(-9999)
# baseline = GradientBoostingRegressor(n_estimators = 100, max_leaf_nodes=None, max_depth=3, random_state=42)

#baseline = HistGradientBoostingRegressor(random_state=42)
#baseline = GradientBoostingRegressor(random_state=42)
#baseline = AdaBoostRegressor(random_state=42)

#baseline = XGBRegressor(n_estimators = 500, random_state = 42, objective= 'rank:ndcg@5')
baseline = XGBRanker(n_estimators = 100, random_state = 42, objective= 'rank:ndcg', eval_metric = 'ndcg@5-')

In [ ]:
# hyper param tuning
# param_dist = {'min_samples_split' : [2, 10_000], 'max_depth' : [3, 15]}
# hp_search = HalvingRandomSearchCV(baseline, param_dist, min_resources=50_000).fit(X_train, y_train)

In [ ]:
# hp_search.best_params_

In [ ]:
groups = X_bal.groupby('srch_id').size().to_frame('size')['size'].to_numpy()

baseline.fit(X_bal.drop(['srch_id'], axis = 1),y_bal, group = groups)

In [ ]:
# pred = baseline.predict(data.drop(['srch_id', 'click_bool', 'gross_bookings_usd', 'booking_bool', 'position', 'target'], axis=1).fillna(-9999))

In [ ]:
#mean_squared_error(data['target'], pred)

In [ ]:
# pred_test = baseline.predict(X_test.drop(['srch_id'], axis=1).fillna(-9999))
# print('MSE', mean_squared_error(y_test, pred_test))

# score = score_predict(X_test, y_test, pred_test)
# print('score', score)

In [ ]:
test_data['pred_score'] = baseline.predict(test_data.drop(['srch_id', 'click_bool', 'gross_bookings_usd', 'booking_bool', 'position', 'target']
                                      , axis=1).fillna(-9999))

In [ ]:
print('MSE', mean_squared_error(test_data['target'], test_data['pred_score']))

In [ ]:
test_data['pred_rank'] = test_data.groupby('srch_id')['pred_score'].rank(ascending=False).astype(int)

In [ ]:
# scoring with ndcg
mean_ndcg = 0
counter = 0
for id in tqdm(test_ids):
    if len(test_data['true_rank'].loc[test_data['srch_id'] == id]) > 1:
        ndcg = ndcg_score([test_data['true_rank'].loc[test_data['srch_id'] == id].astype(int).to_numpy()],
                        [test_data['pred_score'].loc[test_data['srch_id'] == id].to_numpy()], k = 5)

    mean_ndcg += ndcg
    if ndcg < 1.0:
        counter +=1

mean_ndcg = mean_ndcg/len(test_ids)

In [ ]:
mean_ndcg

In [ ]:
plt.bar(range(len(baseline.feature_importances_)), baseline.feature_importances_)
plt.figsize= (4,20)
# plt.xlabel(baseline.get_booster().feature_names, rotation = 90)
plt.xticks(range(len(baseline.feature_importances_)), baseline.get_booster().feature_names,rotation = 90)
plt.tight_layout()
plt.show()


## Predict the test dataset and bring it into requested format

In [ ]:
comp_data = pd.read_csv('data/FE_test_set.csv')
comp_data.drop('Unnamed: 0', axis=1, inplace=True)

# test_data = pd.read_csv('data/test_set_VU_DM.csv')
comp_data.drop('date_time', axis = 1, inplace=True)

In [ ]:
comp_data.head()

In [ ]:
set(comp_data.columns) - set(baseline.get_booster().feature_names)

In [ ]:
comp_data['prediction'] = baseline.predict(comp_data.drop(['srch_id', 'in_train'], axis=1).fillna(-9999))

# test_data['prediction'] = baseline.predict(test_data.drop(['srch_id', 'date_time'], axis=1).fillna(-9999))

In [ ]:
# sort values for final format
comp_data.sort_values(['srch_id', 'prediction'], axis=0, inplace=True, ignore_index=True, ascending=[True, False])

In [ ]:
filename = 'data/predictions/prediction'+str(datetime.now())+'.csv'
comp_data[['srch_id', 'prop_id']].to_csv(filename, index=False)